In [1]:
import pymongo
import time
import random 
import requests
from bs4 import BeautifulSoup 
import os
import pickle
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

linkedinuser = os.environ.get('linkedinuser')
linkedpass = os.environ.get('linkedpass')

sns.set()
%matplotlib inline



In [2]:
# initialize mongo 
onn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.raw_html_db
# db = client.test
collection = db.html

In [3]:
linkedinuser


'alvintomlin@outlook.com'

In [4]:
# Scrape a list of current user agents 
url = 'https://developers.whatismybrowser.com/useragents/explore/operating_system_name/windows/'

def get_agent_list(link,pagenum):
    """
       Use this function to scape and generate a list of windows user agent strings.
       Page num is calculated by visiting the url provided to this function and checking
       to see how many pages are availble to scrape.
    """
    try:
        for i in range(pagenum):
            time.sleep(1)
            r = requests.get(url)
            if r.status_code == 200:
                data = r.content
                soup = BeautifulSoup(data,'html.parser') 
                anchors = [a.get_text() for a in (td.find('a') for td in soup.findAll('td')) if a]
            return anchors
    except Exception as e:
        print(e)

In [5]:
#Choose a random user agent from the scraped list
def generate_user_agent(agent_list):
    try:
        num = random.randint(0,len(agent_list[:25]))
        ua = agent_list[num]
        opts = Options()
        return ua
    except Exception as e:
        print(e)



In [8]:
# This scraper parses and returns the raw html used to get the job links
def login_scrape(uname,pw,agent):

    try:
        pages = []
        offset = 25
        opts = Options()
        opts.add_argument(f"user-agent={agent}")
        driver = webdriver.Chrome('../driver/chromedriver',options=opts)
        driver.get('http://linkedin.com')
        login = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
        time.sleep(random.randrange(1,3))
        login.send_keys(uname)
        password = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
        password.send_keys(pw)   
        submit = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
        submit.send_keys(Keys.ENTER)
        time.sleep(random.randrange(3,8))
 
        
    # scrape the job cards from the job card page
        for i in range(40):
            if i == 0:
                driver.get('https://www.linkedin.com/jobs/search/?geoId=90000049&keywords=data%20scientist&location=Los%20Angeles%20Metropolitan%20Area&sortBy=R')
                action = ActionChains(driver)
                action.send_keys(Keys.TAB * 60)
                action.perform()
                time.sleep(random.randrange(5,8))
                text_contents = BeautifulSoup(driver.page_source)
                job_html = {'job':[driver.page_source]}
                collection.insert_one(job_html)
                pages.extend(text_contents)
            
      
            else:
                driver.get(f'https://www.linkedin.com/jobs/search/?geoId=90000049&keywords=data%20scientist&location=Los%20Angeles%20Metropolitan%20Area&sortBy=R&start={offset}')
#                 time.sleep(random.randrange(1,3))
                action = ActionChains(driver)
                action.send_keys(Keys.TAB * 60)
                action.perform()
                time.sleep(random.randrange(5,8))
                text_contents = BeautifulSoup(driver.page_source)
                job_html = {'job':[driver.page_source]}
                collection.insert_one(job_html)
                pages.extend(text_contents)
                time.sleep(random.randrange(5,8))
                offset += 25            
        
        return pages
    
    except Exception as e:
        print(e)
    finally:
        driver.quit()


In [9]:
#scraping user agents 
data = get_agent_list(url,11)


In [10]:
# grabbing a random user agent
ua = generate_user_agent(data)
ua

list index out of range


In [11]:
#login and scrape job card links from linkedin
jobs = login_scrape(uname=linkedinuser,pw=linkedpass,agent=ua)

In [12]:
#setting up another mongo collection to store the raw html for the links
link_collection = db.job_links
#Parse the raw html and collect individual job links 
links = []
titles = []
for x in range(len(jobs)):
    for a in jobs[x].find_all('a','job-card-search__link-wrapper js-focusable disabled ember-view', href=True):
            links.append(a['href'])
            titles.append(a.text.strip())
            link_collection.insert_one({"link":a.text.strip()})
        
 

In [18]:
#This scraper is run second. It takes the links collected by the previous scrape, visits the links and pull the
#  source code and saves it to a  collection called job_detail in our raw_html_db mongo database
db = client.raw_html_db
job_detail_html = db.job_detail
def job_listings(uname,pw,agent,link_list):

    try:
        job_pages = []
        opts = Options()
        opts.add_argument(f"user-agent={agent}")
        driver = webdriver.Chrome('../driver/chromedriver',options=opts)
        driver.get('http://linkedin.com')
        login = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
        time.sleep(random.randrange(3,8))
        login.send_keys(uname)
        password = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
        time.sleep(random.randrange(3,8))
        password.send_keys(pw)   
        time.sleep(random.randrange(1,8))
        submit = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
        submit.send_keys(Keys.ENTER)
        time.sleep(random.randrange(3,8))
 
        for link in link_list:
            driver.get(f'https://www.linkedin.com{link}')
            time.sleep(random.randrange(2,4))
            link_src = {'link_src':[driver.page_source]}
            job_detail_html.insert_one(link_src)
            job_pages.extend(BeautifulSoup(driver.page_source))
        
        return job_pages
    
    except Exception as e:
        print(e)
    finally:
        driver.quit()

In [19]:
jobs_final = job_listings(uname=linkedinuser,pw=linkedpass,agent=ua,link_list=links)